캐글 설치

In [ ]:
!pip install kaggle

캐글 키 입력

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'sosohangamsung'
os.environ['KAGGLE_KEY'] = 'ee9d9419de6ab91a28058fa4acf092f6'

캐글 데이터 가져오기

In [ ]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

압축 해제

In [ ]:
!unzip -q gtzan-dataset-music-genre-classification.zip

**음향데이터를 librosa를 활용하여 분석을 진행할 수 있게 수치화**
<br>
<br>
y: 소리가 떨리는 세기(진폭)를 시간 순서대로 나열한 것
<br>
sr : sampling rate = 1초당 샘플의 개수, 단위 Hz 또는 KHz

In [ ]:
import librosa

y, sr = librosa.load('Data/genres_original/reggae/reggae.00036.wav')

print(y)
print(len(y))
print('Sampling rate (KHz): %d' % sr)
print('Audio length (seconds): %.2f' % (len(y) /sr))

2D 음파 그래프

In [ ]:
import matplotlib.pyplot as plt
import librosa.display

plt.figure(figsize=(16, 6))
librosa.display.waveplot(y=y, sr=sr)
plt.show()

**Fourier fit_transform**
<br>
<br>
- 시간 영역 데이터를 주파수 영역으로 변경
<br>
- y축 : 주파수(로그 스케일)
<br>
- color축 : 데시벨(진폭)

In [ ]:
import numpy as np

D = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))

print(D.shape)

plt.figure(figsize=(16, 6))
plt.plot(D)
plt.show()

Spectogram
<br>
- 시간에 따른 신호 주파수의 스펙트럼 그래프
- 다른 이름 : Sonographs, Voiceprints, Voicegrams

In [ ]:
DB = librosa.amplitude_to_db(D, ref=np.max)

plt.figure(figsize=(16, 6))
librosa.display.specshow(DB, sr=sr, hop_length=512, x_axis='time', y_axis='log')
plt.colorbar()
plt.show()

**Mel Spectogram**
<br>
- (인간이 이해하기 힘든) Spectogram의 y축을 Mel Scale로 변환한 것(Non-linear transformation)
<br>
- Mel Scale: https://newsight.tistory.com/294

In [ ]:
# 레게

S = librosa.feature.melspectrogram(y, sr=sr)
S_DB = librosa.amplitude_to_db(S, ref=np.max)

plt.figure(figsize=(16, 6))
librosa.display.specshow(S_DB, sr=sr, hop_length=512, x_axis='time', y_axis='log')
plt.colorbar()
plt.show()

In [ ]:
# 클래식
y1, sr1 = librosa.load('Data/genres_original/classical/classical.00036.wav')
y1, _ = librosa.effects.trim(y1)

S1 = librosa.feature.melspectrogram(y1, sr=sr1)
S_DB1 = librosa.amplitude_to_db(S1, ref=np.max)

plt.figure(figsize=(16, 6))
librosa.display.specshow(S_DB1, sr=sr1, hop_length=512, x_axis='time', y_axis='log')
plt.colorbar()
plt.show()

**오디오 특성 추출**
<br>
<br>
음악의 속도(Tempo) 파악하기

In [ ]:
tempo, _ = librosa.beat.beat_track(y, sr=sr)
tempo1, _ = librosa.beat.beat_track(y1, sr=sr1)

print(tempo)
print(tempo1)

**Zero Crossing Rate**
<br>
- 음파가 양에서 음으로 또는 음에서 양으로 바뀌는 비율

In [ ]:
zero_crossings = librosa.zero_crossings(y, pad=False)
zero_crossings1 = librosa.zero_crossings(y1, pad=False)

print(sum(zero_crossings))
print(sum(zero_crossings1))

In [ ]:
# 확대(레게)

n0 = 9000
n1 = 9040

plt.figure(figsize=(16, 6))
plt.plot(y[n0:n1])
plt.grid()
plt.show()

In [ ]:
# 확대(클래식)
plt.figure(figsize=(16, 6))
plt.plot(y1[n0:n1])
plt.grid()
plt.show()

Harmonic and Percussive Components
<br>
<br>
- Harmonics: 사람의 귀로 구분할 수 없는 특징들 (음악의 색깔)
<br>
- Percussives: 리듬과 감정을 나타내는 충격파

In [ ]:
#레게
y_harm, y_perc = librosa.effects.hpss(y)

plt.figure(figsize=(16, 6))
plt.plot(y_harm, color='b')
plt.plot(y_perc, color='r')
plt.show()

In [ ]:
#클래식
y_harm1, y_perc1 = librosa.effects.hpss(y1)

plt.figure(figsize=(16, 6))
plt.plot(y_harm1, color='b')
plt.plot(y_perc1, color='r')
plt.show()

**Spectral Centroid**
<br>
- 소리를 주파수로 표현했을 때, 주파수의 가중평균을 계산하여 소리의 '무게중심'이 어딘지를 알려주는 지표
<br>
- 예를 들어, 블루스 음악은 무게중심이 가운데 놓여있는 반면, 메탈 음악은(끝 부분에 달리기 때문에) 노래의 마지막 부분에 무게중심이 실린다.

In [ ]:
#레게
spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]

frames = range(len(spectral_centroids))

t = librosa.frames_to_time(frames)

import sklearn
def normalize(x, axis=0) :
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

plt.figure(figsize=(16, 6))
librosa.display.waveplot(y, sr=sr, alpha=0.5, color='b')
plt.plot(t, normalize(spectral_centroids), color='r')
plt.show()


In [ ]:
#클래식
spectral_centroids1 = librosa.feature.spectral_centroid(y1, sr=sr1)[0]

frames1 = range(len(spectral_centroids1))

t1 = librosa.frames_to_time(frames1)


plt.figure(figsize=(16, 6))
librosa.display.waveplot(y1, sr=sr1, alpha=0.5, color='b')
plt.plot(t1, normalize(spectral_centroids1), color='r')
plt.show()

**Spectral Rolloff**
<br>
- 신호 모양을 측정한다.
<br>
- 총 스펙트럴 에너지 중 낮은 주파수(85% 이하)에 얼마나 많이 집중되어 있는가

In [ ]:
#레게
spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]

plt.figure(figsize=(16, 6))
librosa.display.waveplot(y, sr=sr, alpha=0.5, color='b')
plt.plot(t, normalize(spectral_rolloff), color='r')
plt.show()

In [ ]:
#클래식
spectral_rolloff1 = librosa.feature.spectral_rolloff(y1, sr=sr1)[0]

plt.figure(figsize=(16, 6))
librosa.display.waveplot(y1, sr=sr1, alpha=0.5, color='b')
plt.plot(t, normalize(spectral_rolloff1), color='r')
plt.show()

**Mel_Frequency Cepstral Coefficients (MFCCs)**
<br>
- MFCCs는 특징들이 작은 집합(약 10-20)으로 스텍프럴 포곡선의 전쳊넉인 모양을 축약하여 보여준다.
<br>
- 사람의 청각 구조를 반영하여 음성 정보 추출
<br>
- https://tech.kakaoenterprise.com/66

In [ ]:
#레게

mfccs = librosa.feature.mfcc(y, sr=sr)
mfccs = normalize(mfccs, axis=1)

print('mean: %.2f' % mfccs.mean())
print('var: %.2f' % mfccs.var())

plt.figure(figsize=(16, 6))
librosa.display.specshow(mfccs, sr=sr, x_axis='time')
plt.show()

In [ ]:
#클래식
mfccs1 = librosa.feature.mfcc(y1, sr=sr1)
mfccs1 = normalize(mfccs1, axis=1)

print('mean: %.2f' % mfccs1.mean())
print('var: %.2f' % mfccs1.var())

plt.figure(figsize=(16, 6))
librosa.display.specshow(mfccs1, sr=sr1, x_axis='time')
plt.show()

Chroma Frequencies
<br>
- 크로마 특징은 음악의 흥미롭고 강렬한 표현이다.
<br>
- 크로마는 인간 청각이 옥타브 차이가 나는 주파수를 가진 두 음을 유사음으로 인지한다는 음악이론에 기반한다
<br>
- 모든 스펙트럼을 12개의 Bin으로 표현한다.
- 12개의 Bin은 옥타브에서 12개의 각기 다른 반음(Semitones=Chroma)을 의미한다.

In [ ]:
#레게

chromagram = librosa.feature.chroma_stft(y, sr=sr, hop_length=512)

plt.figure(figsize=(16, 6))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=512)
plt.show()

In [ ]:
#클래식

chromagram1 = librosa.feature.chroma_stft(y1, sr=sr1, hop_length=512)

plt.figure(figsize=(16, 6))
librosa.display.specshow(chromagram1, x_axis='time', y_axis='chroma', hop_length=512)
plt.show()

**데이터 프레임 불러오기**

In [ ]:
import pandas as pd

df = pd.read_csv('Data/features_3_sec.csv')

df.head()

전처리

In [ ]:
X = df.drop(columns=['filename', 'length', 'label'])
y = df['label']

scaler = sklearn.preprocessing.MinMaxScaler()
np_scaled = scaler.fit_transform(X)

X = pd.DataFrame(np_scaled, columns=X.columns)

X.head()

train, test set 분리

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

random forest 학습 및 평가

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

print('정확도 : %.2f' % metrics.accuracy_score(y_test, y_pred))

xgb boost 학습 및 평가

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
xgb.fit(X_train, y_train)

y_preds = xgb.predict(X_test)

print('Accuracy: %.2f' % accuracy_score(y_test, y_preds))

confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(16, 9))
sns.heatmap(
    cm,
    annot=True,
    xticklabels=["blues", "classical", "country", "disco", "gipgop", "jazz", "meta", "pop", "reggae", "rock"],
    yticklabels=["blues", "classical", "country", "disco", "gipgop", "jazz", "meta", "pop", "reggae", "rock"]
)
plt.show()

In [ ]:
df_importance = pd.DataFrame([X_test.columns, forest.feature_importances_]).T
df_importance.sort_values(1, ascending=False).head(10)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_preds)

plt.figure(figsize=(16, 9))
sns.heatmap(
    cm,
    annot=True,
    xticklabels=["blues", "classical", "country", "disco", "gipgop", "jazz", "meta", "pop", "reggae", "rock"],
    yticklabels=["blues", "classical", "country", "disco", "gipgop", "jazz", "meta", "pop", "reggae", "rock"]
)
plt.show()

feature_importances

In [ ]:
df_importance = pd.DataFrame([X_test.columns, xgb.feature_importances_]).T
df_importance.sort_values(1, ascending=False).head(10)

**딮러닝 모델 학습**

In [ ]:
!pip install -U keras-tuner

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
import tensorflow as tf
import IPython
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import kerastuner as kt

In [ ]:
y_train = pd.DataFrame(y_train)

In [ ]:
y_test = pd.DataFrame(y_test)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
y_train_encoded = enc.fit_transform(y_train)
y_test_encoded = enc.fit_transform(y_test)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu'))
Dropout(0.5)
model.add(Dense(512, activation='relu'))
Dropout(0.5)
model.add(Dense(512, activation='relu'))
Dropout(0.5)
model.add(Dense(512, activation='relu'))
Dropout(0.5)
model.add(Dense(512, activation='relu'))
Dropout(0.5)
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=500)

In [ ]:
model.evaluate(X_test,  y_test_encoded)

In [ ]:
df_30 = pd.read_csv('Data/features_30_sec.csv', index_col='filename')

labels = df_30[['label']]
df_30 = df_30.drop(columns=['length', 'label'])

df_30_scaled = sklearn.preprocessing.scale(df_30)

df_30 = pd.DataFrame(df_30_scaled, columns=df_30.columns)

df_30.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(df_30)

sim_df = pd.DataFrame(similarity, index=labels.index, columns=labels.index)

sim_df.head()

In [ ]:
def find_similar_songs(name, n=5):
    series = sim_df[name].sort_values(ascending=False)

    series = series.drop(name)

    return series.head(n).to_frame()

find_similar_songs('rock.00000.wav')
